In [7]:
import sys
import torch
import speechbrain as sb
from torch.utils.data import DataLoader
from hyperpyyaml import load_hyperpyyaml
from speechbrain.utils.parameter_transfer import Pretrainer

In [4]:
hparams_file = '../hparams/conformer_medium.yaml'

In [5]:
with open(hparams_file) as fin:
    hparams = load_hyperpyyaml(fin)

In [6]:
model = hparams['model']

In [11]:
pretrained_model = '../Inference/pretrained-model-src/kspon/asr.ckpt'
pretrain = Pretrainer(collect_in='model_local', loadables={'model': model}, paths={'model': pretrained_model})

In [13]:
pretrain.collect_files()
pretrain.load_collected()